In [3]:
%matplotlib inline

from scipy.odr import *
from scipy.stats import *
import numpy as np
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import ast
from multiprocessing import Pool, cpu_count

import scipy

from IPython import display
from matplotlib.patches import Rectangle

from sklearn.metrics import mean_squared_error
import json

import scipy.stats as st
from sklearn.metrics import r2_score


from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt

import copy

from sklearn.model_selection import LeaveOneOut, LeavePOut

from multiprocessing import Pool
import cv2

In [4]:
df_filtered = pd.read_pickle("PklData/dfFiltered_LSTM.pkl")

In [5]:
df_filtered.head()

,userID,Timestamp,Current_Task,Task_amount,TaskID,VersionID,RepetitionID,Actual_Data,Is_Pause,Image
10578,15,1547132393277,2,680,12,2,0,True,False,"[0, 224, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0..."
10591,15,1547132393806,2,680,12,2,0,True,False,"[0, 224, 1, 0, 0, 161, 4, 2, 4, 3, 4, 0, 182, ..."
10592,15,1547132393844,2,680,12,2,0,True,False,"[0, 224, 1, 0, 0, 161, 4, 2, 4, 3, 4, 0, 182, ..."
10593,15,1547132393884,2,680,12,2,0,True,False,"[0, 224, 1, 0, 0, 161, 4, 2, 4, 3, 4, 0, 182, ..."
10594,15,1547132393924,2,680,12,2,0,True,False,"[0, 224, 1, 0, 0, 161, 4, 2, 4, 3, 4, 0, 182, ..."


In [6]:
df_filtered.Image = df_filtered.Image.apply(lambda x: x.reshape(27, 15))
df_filtered.Image = df_filtered.Image.apply(lambda x: x.clip(min=0, max=255))
df_filtered.Image = df_filtered.Image.apply(lambda x: x.astype(np.uint8))
df_filtered["ImageSum"] = df_filtered.Image.apply(lambda x: np.sum(x))

In [7]:
#LSTMs new Blob detection
def detect_blobs(image):
    #image = image.reshape(27, 15)
    large = np.ones((29,17), dtype=np.uint8)
    large[1:28,1:16] = image
    temp, thresh = cv2.threshold(cv2.bitwise_not(large), 200, 255, cv2.THRESH_BINARY)
    im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = [a for a in contours if cv2.contourArea(a) > 5 and cv2.contourArea(a) < 255]
    lstBlob  = []
    lstMin = []
    lstMax = []
    count = 0
    return len(contours) > 0

In [8]:
%%time
pool = Pool(cpu_count() - 2)
temp_blobs = pool.map(detect_blobs, df_filtered.Image)
pool.close()

CPU times: user 3.41 s, sys: 399 ms, total: 3.81 s
Wall time: 6.57 s


In [9]:
df_filtered["ContainsBlobs"] = temp_blobs

In [10]:
#Label if knuckle or finger
def f(row):
    if row['TaskID'] < 17:
        #val = "Knuckle"
        val = 0
    elif row['TaskID'] >= 17:
        #val = "Finger"
        val = 1
    return val
df_filtered['InputMethod'] = df_filtered.apply(f, axis=1)

In [11]:
def cast_to_int(x):
    if type(x) == int:
        return x
    x = str(x).replace(",", ".")
    return int(float(x))

df_filtered.Timestamp = df_filtered.Timestamp.map(cast_to_int)

In [12]:
%%time
UserIDs = []
TaskIDs = []
VersionIDs = []
Blobs = []
for userID in df_filtered.userID.unique():
    print(userID)
    for TaskID in df_filtered[df_filtered.userID == userID].TaskID.unique():
        for VersionID in df_filtered[(df_filtered.userID == userID) & (df_filtered.TaskID == TaskID)].VersionID.unique():
            first_blob = -1
            last_blob = -1
            for index, row in df_filtered[(df_filtered.userID == userID) & (df_filtered.TaskID == TaskID) & (df_filtered.VersionID == VersionID)].iterrows():
                if row.ContainsBlobs:
                    last_blob = row.Timestamp
                    if first_blob == -1:
                        first_blob = row.Timestamp
            if first_blob >= 0 and last_blob >= 0:
                UserIDs.append(userID)
                TaskIDs.append(TaskID)
                VersionIDs.append(VersionID)
                Blobs.append(df_filtered[(df_filtered.userID == userID) & (df_filtered.TaskID == TaskID) & (df_filtered.VersionID == VersionID) & (df_filtered.Timestamp >= first_blob) & (df_filtered.Timestamp <= last_blob)].Image.values)

15
3
7
16
5
9
17
14
6
10
2
4
12
1
13
8
11
CPU times: user 4min 3s, sys: 126 ms, total: 4min 3s
Wall time: 4min 3s


In [13]:
UserIDs = np.array(UserIDs, dtype=np.int64)
TaskIDs = np.array(TaskIDs, dtype=np.int64)
VersionIDs = np.array(VersionIDs, dtype=np.int64)

In [14]:
df_lstm = pd.DataFrame()
df_lstm["userID"] = UserIDs
df_lstm["TaskID"] = TaskIDs
df_lstm["VersionID"] = VersionIDs
df_lstm["Blobs"] = Blobs
df_lstm.head()

,userID,TaskID,VersionID,Blobs
0,15,12,2,"[[[0, 224, 1, 0, 0, 161, 4, 2, 4, 3, 4, 0, 182..."
1,15,12,4,"[[[1, 224, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,..."
2,15,12,7,"[[[1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 4, 2, 0, 0..."
3,15,12,8,"[[[0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0..."
4,15,12,10,"[[[0, 0, 0, 1, 1, 1, 5, 3, 0, 0, 0, 0, 0, 0, 0..."


In [15]:
df_lstm.to_pickle("PklData/df_lstm.pkl")